## Python Module: Add a Subnet to a Network

This notebook demonstrates how to use the OpenStack python API to add a subnet to a network. 

The method below is included can be used in any notebook by importing the chi library using "import chi". 

#### Related Modules

- [Delete Subnet](./delete_subnet.ipynb)
- [Get Subnet by Name](./get_subnet_by_name.ipynb)
- [Get Network by Name](./get_network_by_name.ipynb)
- [Create Network](./create_network.ipynb)

#### Arguments

Most networks will only requrie one argument which names the network:

- network_name: The desired name for the new network

OpenFlow, stitched, and multi-vlan networks require additional arguments: 

- of_controller_ip (optional): OpenFlow controller IP. Only required for OpenFlow networks.
- of_controller_port (optional): OpenFlow controller network port. Only required for OpenFlow networks.
- vswitch_name (optional): The name of the vswtich to attach additional VLANs to the a single swtich.
- provider (optional): OpenStack provider network. Defaults to 'physnet1', stitched networks need to specifiy the appopriate stitching provider.

#### OpenStack API Method 

In [ ]:
import json
import os
import chi

neutron = chi.neutron()
    
def add_subnet(subnet_name, network_name, cidr='192.168.1.0/24'):
    network=get_network_by_name(name=network_name)
    network_id=network['id']

    #Add Subnet
    body_create_subnet = {'subnets': [{'cidr': cidr,
                                       'ip_version': 4, 
                                       'network_id': network_id,
                                       'name': subnet_name,
                                      }]
                          }
    subnets = neutron.create_subnet(body=body_create_subnet)
    
    subnet = get_subnet_by_name(subnet_name)
    return subnet

def get_network_by_name(name):
    nets = neutron.list_networks()['networks']
    for net in nets:
        if net['name'] == name:
            return net
    raise RuntimeError("couldn't find net with name '{}'".format(name))
    
def get_subnet_by_name(name):
    subnet=None
    for s in neutron.list_subnets()['subnets']:
        if s['name'] == name:
            if subnet != None:
                print("Found multiple subnets with name " + str(name))
                return
            subnet = s
    
    return subnet



#### Example

In [ ]:
import chi

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'

# Get your username to uniquely label your resources and reservations
username = os.getenv("USER")

#Set the network name to "<username>Network1" 
network_name = username+'Network1'
subnet_name = username+'Subnet1'

#Get a network by name
subnet = add_subnet(subnet_name, network_name, cidr='192.168.100.0/24')
    
#Print the network info
print(json.dumps(subnet, indent=2))

#Get the network's ID
subnet_id = subnet['id']
print('Subnet ID: ' + subnet_id)
